In [1]:
# Does not include training, only testing
%load_ext autoreload
%autoreload 2
%aimport RNN_model, RNN_apply_ind

In [ ]:
'''
If not have run before
'''
import RNN_apply_ind, os, json, argparse

folder_path = r'C:\Users\qilei.zhang\OneDrive - Frontier Airlines\Documents\Data\USconti'
seats_file_name = r'\Schedule_Monthly_Summary_Report_Conti.csv'
perf_file_name = r'\Airline_Performance_Report_Conti.csv'
apply_file_name = '\Schedule_Monthly_Summary_2023Q1234.csv'
# Load parameters from the JSON file.
if not os.path.exists('parameters.json'):
    print("parameters.json does not exist, Find the file and put it in the same folder as this file")
with open('parameters.json', 'r') as f:
    args = argparse.Namespace(**json.load(f))

RNN_apply_ind.main_apply(args, folder_path, seats_file_name, perf_file_name, apply_file_name)

In [19]:
'''
quick look at the result
'''
import pandas as pd
from RNN_apply_ind import DataAna

apply_filename='./data/applying_data.csv'
ana_df_name = "./results/data_to_ana_apply.csv"
orig_df = pd.read_csv(apply_filename)

ana = DataAna(ana_df_name)
ana.merge_previous_data(orig_df)

while True:
    user_input = input("Enter airline and route, separated by comma, or 'c' to exit: ")
    if user_input.lower() == 'c':
        break
    try:
        airline, route = user_input.split(',')
        airline = airline.strip()  # remove possible leading/trailing whitespaces
        route = route.strip()  # remove possible leading/trailing whitespaces
        ana.plot_prediction(airline, route)
    except ValueError:
        print("Invalid input, please enter the airline and route separated by a comma or 'continue' to proceed.")

In [14]:
'''
Add new performance data and seats data to original data
'''

import pandas as pd

folder_path = r'C:\Users\qilei.zhang\OneDrive - Frontier Airlines\Documents\Data\USconti'
seats_file_name = r'\Schedule_Monthly_Summary_Report_Conti.csv'
perf_file_name = r'\Airline_Performance_Report_Conti.csv'

seats_file_new = r'\Schedule_Monthly_Summary_2023Q1.csv'
perf_file_new = r'\Airline_Performance_Report_USconti_2023Q1.csv'

seats_df = pd.read_csv(folder_path + seats_file_name)
perf_df = pd.read_csv(folder_path + perf_file_name)

seats_df_new = pd.read_csv(folder_path + seats_file_new)
perf_df_new = pd.read_csv(folder_path + perf_file_new)


# Check the column names are the same othewise not proceed
if not (seats_df.columns == seats_df_new.columns).all():
    print("Column names are not the same, please check the data")
    raise ValueError

if not (perf_df.columns == perf_df_new.columns).all():
    print("Column names are not the same, please check the data")
    raise ValueError

# Check the data types are the same othewise not proceed
if not (seats_df.dtypes == seats_df_new.dtypes).all():
    print("Column types are not the same, please check the data")
    raise ValueError

if not (perf_df.dtypes == perf_df_new.dtypes).all():
    print("Column types are not the same, please check the data")
    raise ValueError

seats_df = pd.concat([seats_df, seats_df_new])
perf_df = pd.concat([perf_df, perf_df_new])

seats_df.to_csv(folder_path + seats_file_name, index=False)
perf_df.to_csv(folder_path + perf_file_name, index=False)


In [17]:
from RNN_model import calculate_quarters

calculate_quarters(3, 10, "Q1 2023")

('Q2 2022', 'Q1 2020', 'Q4 2020')